In [29]:
import os
import re


In [265]:
class parse_metrics(object):
    def __init__(self,metrics_file):
        assert os.path.exists(metrics_file)
        self.filename = metrics_file
        with open(self.filename,'r') as f:
            self.file = f.readlines()
        self.total_pos_and_neg()    
        self.metrics_dict = {}  
        n=1
        while(self.extract_metrics_of_epoch(n)):
            print("Parsing Epoch: {}".format(n))
            self.metrics_dict[n] = self.parse_metrics_of_epoch(n)
            n+=1
        self.best_epoch = self.extract_best_epoch()    
            
    
    def extract_best_epoch(self):
        last_line  = self.file[-1]
        match = re.search('from epoch [0-9]+',last_line)
        if match:
            match = match.group()
        else:
            return("Model has not finished training/trained")
        best_epoch = int(match.lstrip('from epoch'))
        return best_epoch
    
    def extract_metrics_of_epoch(self,n):
        for i,line in enumerate(self.file):
            match = re.search('Epoch {}:'.format(str(n)),line)
            if match:
                break
        if i==(len(self.file)-1):   #Went to the end of the file with no matches found
            print("No match found for Epoch: {} \n".format(n))
            return
        else:
            return self.file[i:i+4]     
             
    def parse_metrics_of_epoch(self,n):
        metrics_dict = {}
        if n in self.metrics_dict:
            return self.metrics_dict[n]
        
        
        metrics = self.extract_metrics_of_epoch(n)
        if metrics:
            metrics_dict['Epoch'] = n
            line3 = metrics[2]
            metrics_dict['auroc'] = float(line3.split('\t')[1].lstrip('auROC:'))
            metrics_dict['auprc'] = float(line3.split('\t')[2].strip().lstrip('auPRC:'))
            line4 = metrics[3]
            match = re.search('FDR: [0-9.% | ]+',line4).group()
            match = match.lstrip('FDR:').split('|')
            recall_vals = [float(_.strip().rstrip('%')) for _ in match]
            metrics_dict['recall_5'] = recall_vals[0]
            metrics_dict['recall_10'] = recall_vals[1]
            metrics_dict['recall_25'] = recall_vals[2]
            metrics_dict['recall_50'] = recall_vals[3]
            return metrics_dict
            
        
        return 
    
    def total_pos_and_neg(self):
        metrics = self.extract_metrics_of_epoch(1)
        pos,neg = re.search('Num Positives: [0-9]+',metrics[-1]).group(),\
        re.search('Num Negatives: [0-9]+',metrics[-1]).group()
        pos = int(pos.split(':')[-1])
        neg = int(neg.split(':')[-1])
        self.pos = pos
        self.neg = neg
        
        
        
        

In [266]:
path_to_metrics_file = '/mnt/lab_data/kundaje/manyu/DapSeq_models/seq_meth_C_models/logdir_DEL2/metrics.log'
file_parser = parse_metrics(path_to_metrics_file)

Parsing Epoch: 1
Parsing Epoch: 2
Parsing Epoch: 3
Parsing Epoch: 4
Parsing Epoch: 5
Parsing Epoch: 6
Parsing Epoch: 7
Parsing Epoch: 8
Parsing Epoch: 9
Parsing Epoch: 10
Parsing Epoch: 11
Parsing Epoch: 12
Parsing Epoch: 13
Parsing Epoch: 14
Parsing Epoch: 15
Parsing Epoch: 16
No match found for Epoch: 17 



In [267]:
file_parser.metrics_dict[file_parser.best_epoch]


{'Epoch': 9,
 'auprc': 0.551,
 'auroc': 0.979,
 'recall_10': 18.4,
 'recall_25': 42.7,
 'recall_5': 6.3,
 'recall_50': 56.9}

In [255]:
file_parser.__dict__.keys()

dict_keys(['pos', 'metrics_dict', 'best_epoch', 'filename', 'file', 'neg'])

In [223]:
metrics[-1]

'Recall at 5% | 10% | 25% | 50% FDR: 0.5% | 0.5% | 19.3% | 44.6%\tNum Positives: 921\t Num Negatives: 116328\n'

In [230]:
pos,neg = re.search('Num Positives: [0-9]+',metrics[-1]).group(),re.search('Num Negatives: [0-9]+',metrics[-1]).group()

In [232]:
pos,neg = re.search('Num Positives: [0-9]+',metrics[-1]).group(),re.search('Num Negatives: [0-9]+',metrics[-1]).group()
pos = int(pos.split(':')[-1])
neg = int(neg.split(':')[-1])
print(pos,neg)

921 116328


In [205]:
match = re.search('FDR: [0-9.% | ]+',line).group()
match = match.lstrip('FDR:').split('|')

In [198]:
[float(_.strip().rstrip('%')) for _ in match]

[5.1, 14.2, 40.6, 71.3]